In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

Saving troll_poem.csv to troll_poem.csv
User uploaded file "troll_poem.csv" with length 19179 bytes


In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 895 kB 36.3 MB/s 
     |████████████████████████████████| 3.3 MB 26.5 MB/s 
     |████████████████████████████████| 596 kB 46.1 MB/s 


In [ ]:
import pandas as pd
import numpy as np

import random
import time
import datetime

import torch
from transformers import GPT2TokenizerFast, BertTokenizerFast, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelWithLMHead
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler

In [ ]:
root = pd.read_csv('/content/troll_poem.csv')

In [ ]:
root.content = root.content.str.replace('.', ' . ')
root.content = root.content.str.replace(' <br>', '\1')

In [ ]:
root.content.apply(lambda x: x.split()).apply(len).describe()

count     37.000000
mean      53.891892
std       25.799207
min       13.000000
25%       33.000000
50%       52.000000
75%       69.000000
max      101.000000
Name: content, dtype: float64

In [ ]:
RANDOM_SEED = 821
BATCH_SIZE = 2
EPOCHS = 10
MAX_LEN = 300

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")

special_tokens_dict = {
    'bos_token': '<BOS>', 
    'eos_token': '<EOS>', 
    'pad_token': '<PAD>'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.add_tokens('\1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


1

In [ ]:
class PoemDataset(Dataset):
    def __init__(self, data, tokenizer, gpt2_type='gpt2', max_length=MAX_LEN):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []
        
        for i in data:
            encodings_dict = tokenizer('<BOS>' + i + '<EOS>',
                                     truncation=True,
                                     max_length=max_length,
                                     padding='max_length')

            self.input_ids.append(torch.tensor(encodings_dict['input_ids'][1:]))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask'][1:]))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]
        
poem_dataset = PoemDataset(root['content'].values, tokenizer, max_length=MAX_LEN)

In [ ]:
def train_val_split(split, dataset):
    train_size = int(split * len(dataset))
    val_size = len(dataset) - train_size
    return train_size, val_size
poem_line_train_size, poem_line_val_size = train_val_split(0.9, poem_dataset)
poem_line_train_dataset, poem_line_val_dataset = random_split(poem_dataset, [poem_line_train_size, poem_line_val_size])

In [ ]:
torch.cuda.manual_seed_all(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
poem_train_dataloader = DataLoader(poem_line_train_dataset,
                              sampler=RandomSampler(poem_line_train_dataset),
                              batch_size=BATCH_SIZE)

poem_val_dataloader = DataLoader(poem_line_val_dataset,
                            sampler=SequentialSampler(poem_line_val_dataset),
                            batch_size=BATCH_SIZE)

In [ ]:
# helper function for logging time
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

# hyperparameters
learning_rate = 5e-4
eps = 1e-8
warmup_steps = 100

# create text generation seed prompt
device = torch.device('cuda')

prompt = "오늘"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

In [ ]:
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=MAX_LEN).from_pretrained("kykim/gpt3-kor-small_based_on_gpt2", output_hidden_states=True)

poem_model = GPT2LMHeadModel.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2",
                                             config=configuration)

poem_model.resize_token_embeddings(len(tokenizer))

poem_model.cuda()
optimizer = AdamW(poem_model.parameters(), lr=learning_rate, eps=eps)

total_steps = len(poem_train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

start_time = time.time()
poem_model = poem_model.to(device)

for epoch_i in range(0, EPOCHS):

    print(f'Epoch {epoch_i + 1} of {EPOCHS}')

    t0 = time.time()
    total_train_loss = 0
    poem_model.train()

    for step, batch in enumerate(poem_train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        poem_model.zero_grad()        

        outputs = poem_model(b_input_ids,
                             labels=b_labels,
                             attention_mask=b_masks,
                             token_type_ids=None)

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(poem_train_dataloader)       
    training_time = format_time(time.time() - t0)

    print(f'Average Training Loss: {avg_train_loss}. Epoch Training Time: {training_time}')

    t0 = time.time()

    poem_model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in poem_val_dataloader:
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():        

            outputs  = poem_model(b_input_ids,
                                  attention_mask=b_masks,
                                  labels=b_labels)

            loss = outputs[0]  

        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(poem_val_dataloader)


    print(f'Average Validation Loss: {avg_val_loss}')

print(f'Total Training Time: {format_time(time.time()-start_time)}')


Epoch 1 of 10
Average Training Loss: 8.575891606947955. Epoch Training Time: 0:00:07
Average Validation Loss: 7.25028920173645
Epoch 2 of 10
Average Training Loss: 7.680476104511934. Epoch Training Time: 0:00:07
Average Validation Loss: 6.3503577709198
Epoch 3 of 10
Average Training Loss: 5.674006546244902. Epoch Training Time: 0:00:07
Average Validation Loss: 4.1422858238220215
Epoch 4 of 10
Average Training Loss: 2.988982488127316. Epoch Training Time: 0:00:06
Average Validation Loss: 2.4552539587020874
Epoch 5 of 10
Average Training Loss: 1.1463229375727035. Epoch Training Time: 0:00:07
Average Validation Loss: 2.0547581911087036
Epoch 6 of 10
Average Training Loss: 0.6561871889759513. Epoch Training Time: 0:00:06
Average Validation Loss: 2.1788527965545654
Epoch 7 of 10
Average Training Loss: 0.43205952644348145. Epoch Training Time: 0:00:06
Average Validation Loss: 2.269427180290222
Epoch 8 of 10
Average Training Loss: 0.28767315342145805. Epoch Training Time: 0:00:06
Average Vali

In [ ]:
import re

In [ ]:
poem_model.eval()

def poet_generator(poet, MAX_LEN=70):
    prompt = f"<BOS> {poet}"
    generated = torch.tensor(tokenizer.encode(prompt)[1:]).unsqueeze(0)
    generated = generated.to(device)

    sample_outputs = poem_model.generate(
                                    generated, 
                                    do_sample=True,   
                                    top_k=30, 
                                    max_length=MAX_LEN,
                                    top_p=0.95, 
                                    num_return_sequences=3,
                                    repetition_penalty=1.5
                                    )

    for i, sample_output in enumerate(sample_outputs):
        result = "{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True))
        result = re.sub(r"<br>", "\n", result)
        print(result)

In [ ]:
poet_generator("한 편의 시는")

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


0: 한 편의 시는 어떤 위로가 되고 용기가 되는 걸까. 시적 화두는 여전히 남아 있다. 고통은 삶의 최종 장이다. 고통을 이기기 위해서는 아픔도 이겨야 한다. 죽음으로 갚지 않아도 된다. 이 시를 읽을 때 쯤이면 나도 저 글에 닿았을 텐데 하는 생각이 스쳐 지나갔다. 어느새 내 손가락의 움직임이 두


1: 한 편의 시는 내 시를 애워싸고 너와 누우면 너는 사라진다. 영원의 이름 석 자가 남은 동안 너를 맞이하겠지. 겨울잠을 자던 너의 등뒤에서 빛이 날 때면면 조각들이 춤추는 것을 보면 천 번을 접어야지만 할 달음박질처럼 접어지네요


2: 한 편의 시는 삶을 살아가는 동안 필요한 것을 내어주고 얻지 못한 것들을 건네는 말들이겠지. 시와 함께 나누었던 행복은 이내 사라져버리고 맙니다. 남은 것들은 그저 빈자리의 시인이 될 뿐이고 영원히 볼 수 없는 것들일뿐이에요. 박노해에게서 배운 것도 별로 없이 그냥 지나버린 시간들 속에서 저는




## 모델 저장

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
home_directory = '/gdrive/My Drive/'
torch.save(poem_model.state_dict(), home_directory + 'poem_model.pth')

## 모델 불러오기 및 실행

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import torch
import re

In [ ]:
model = torch.load(home_directory + 'poem_model.pth')

In [ ]:
def poet_generator(poet, MAX_LEN=70):
    prompt = f"<BOS> {poet}"
    generated = torch.tensor(tokenizer.encode(prompt)[1:]).unsqueeze(0)
    generated = generated.to(device)

    sample_outputs = poem_model.generate(
                                    generated, 
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length=MAX_LEN,
                                    top_p=0.95, 
                                    num_return_sequences=3,
                                    repetition_penalty=3.0
                                    )

    for i, sample_output in enumerate(sample_outputs):
        result = "{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True))
        result = re.sub(r"<br>", "\n", result)
        print(result)

In [ ]:
poet_generator("안녕 나의 친구.", 80)

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


0: 안녕 나의 친구.! 빛은 상처에 닿아 빛이 되고 달이 떠오르면 또다른 별빛이 된다 우리는 이 경험을 통해서 우리들의 삶을 긍정으로 가득 채우려 애쓴다 그러나 사실은 아무것도 할 수 없는 상태에 직면해버리고 맙니다 우리가 잘 하고 있는지조차 깨닫지 못하고 그 순간에는 그저 방탕한 생활뿐이죠 하지만 우리의 역사를 돌아보면 그들의 업적은 눈부시게 빛나는데요 어떤


1: 안녕 나의 친구.뿐이고 터닝 포인트는 보이지 않아요 우리들의 고향은 어디나 마찬가지예요 그리고 그 곳은 영원히 잊혀지게 마련이지요 그러나 이 글의 주인공처럼 다시 피어오를 꽃을 위해 우리는 노력해야 해요 그래야 영원하겠지요 우리가 살고 있는 동네도 이제는 그들의 손짓에 의해 점령당하고 있어요 우리 모두 잘 살아보아요 저쪽에서 우리를 향해 다가오는 사람이 있겠지 하지만 언제까지 우리의 땅


2: 안녕 나의 친구.하고 싶은 말 그대와 내가 나눌 말의 무게를 이기며 달려가는 말은 아무것도 할 수 없는 사람뿐이겠지, 모든 걸 놓아버리고 오직 내 발만 동동 굴리는 것만 남은 사람은 영원히 알 수가 없지 않을까 하는 생각에 주저하는 제가 볼 땐 슬픈 하루일 수도 희망적인 내일의 일수도 있겠군요 허허 그런 까닭에 저는 항상 지쳐있고 허무맹랑한


